## Setting up Plotting Functions

In [1]:
from ipycanvas import Canvas


canvas_width = 500
canvas_height = 500

def setbounds(a,b,c,d):
    global xmin, xmax, ymin, ymax
    xmin = a
    ymin = b
    xmax = c
    ymax = d

def transform(x,y):
    return (x - xmin) / (xmax - xmin) * canvas_width, (y - ymin) / (ymax - ymin) * canvas_height 

def draw():
    a = x1[0] - 0.5 * w1[0]
    b = y1[0] - 0.5 * h1[0]
    c = x1[0] + 0.5 * w1[0]
    d = y1[0] + 0.5 * h1[0]
    e = x2[0] - 0.5 * w2[0]
    f = y2[0] - 0.5 * h2[0]
    g = x2[0] + 0.5 * w2[0]
    h = y2[0] + 0.5 * h2[0]
    
    p1 = transform(a,b)
    p2 = transform(c,d)
    q1 = transform(e,f)
    q2 = transform(g,h)
    
    canvas = Canvas(width=canvas_width, height=canvas_height)
    
    canvas.fill_style = "#000"
    canvas.fill_rect(0,0,canvas_width, canvas_height)
    
    canvas.fill_style = "#F559"
    canvas.fill_rect(p1[0], p1[1], p2[0] - p1[0], p2[1] - p1[1])
    
    canvas.fill_style = "#55F9"
    canvas.fill_rect(q1[0], q1[1], q2[0] - q1[0], q2[1] - q1[1])
    
    return canvas

## Definitions

In [2]:
from gekko import GEKKO

g = GEKKO(remote=False)
g.options.SOLVER = 3

max_ratio = 1.1

def SMAX(x, y, sroot: GEKKO, tau = 0.01):
    return 0.5 * (x + y + sroot( (x - y)**2 + 4 * tau * tau ))
    
def THIN(w, h = 1):
    return 2*w*h/(w*w + h*h)

## Initialization

In [3]:
A1 = 5
A2 = 16

die_width = 7
die_height = 7

x1, y1, w1, h1 = g.Var(lb=0, ub=die_width), g.Var(lb=0, ub=die_height), g.Var(lb=0, ub=die_width), g.Var(lb=0, ub=die_height)
x2, y2, w2, h2 = g.Var(lb=0, ub=die_width), g.Var(lb=0, ub=die_height), g.Var(lb=0, ub=die_width), g.Var(lb=0, ub=die_height)

x1.value, y1.value, w1.value, h1.value = [2], [4], [1], [5]
x2.value, y2.value, w2.value, h2.value = [2], [2], [4], [4]


In [4]:
setbounds(-1, -1, die_width + 1, die_height + 1)
draw()

Canvas(width=500)

## Model Definition

In [5]:
# Model
g.Equation(w1 * h1 == A1)
g.Equation(w2 * h2 == A2)

g.Equation(THIN(w1, h1) >= THIN(max_ratio))
g.Equation(THIN(w2, h2) >= THIN(max_ratio))

t1 = (x1 - x2)**2 - 0.25 * (w1 + w2)**2
t2 = (y1 - y2)**2 - 0.25 * (h1 + h2)**2
g.Equation(SMAX(t1, t2, g.sqrt) >= 0)

g.Equation(x1 - 0.5 * w1 >= 0)
g.Equation(x2 - 0.5 * w2 >= 0)
g.Equation(y1 - 0.5 * h1 >= 0)
g.Equation(y2 - 0.5 * h2 >= 0)
g.Equation(x1 + 0.5 * w1 <= die_width)
g.Equation(x2 + 0.5 * w2 <= die_width)
g.Equation(y1 + 0.5 * h1 <= die_height)
g.Equation(y2 + 0.5 * h2 <= die_height)

g.Obj((x1-x2)**2 + (y1-y2)**2)

## Solve

In [6]:
g.solve(disp=True)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  0
   Constants    :  0
   Variables    :  19
   Intermediates:  0
   Connections  :  0
   Equations    :  14
   Residuals    :  14
 
 Number of state variables:    19
 Number of total equations: -  13
 Number of slack variables: -  11
 ---------------------------------------
 Degrees of freedom       :    -5
 
 * Warning: DOF <= 0
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse

In [7]:
draw()

Canvas(width=500)